<a href="https://colab.research.google.com/github/AhmedCoolProjects/ESI/blob/main/Bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BY AHMED BARGDY

In [1]:
# Préparation de l’environnement du travail

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, classification_report

from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional

In [2]:
def handle_bad_lines(line):
    print(f"Skipping line: {line}")

data_url = "https://firebasestorage.googleapis.com/v0/b/esi-school-resources.appspot.com/o/text_mining%2Ftps%2FIMDB%20Dataset.csv?alt=media&token=005703f2-e4f5-4d48-8c82-77dabb6dc938"

# Chargement et Exploration du Jeu de Données
df_data = pd.read_csv(data_url, header=0, encoding='utf-8', engine="python", on_bad_lines=handle_bad_lines)


# Exploration du dataset.

In [3]:
# Afficher les premières lignes du dataset :
df_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
# Afficher des informations sur les variables :
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [5]:
# Afficher des statistiques descriptives :
df_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [6]:
#  les valeurs uniques dans les colonnes catégorielles :
print(f"Unique values in sentiment: {df_data['sentiment'].unique()}")


Unique values in sentiment: ['positive' 'negative']


In [7]:
#  la distribution des valeurs dans les colonnes catégorielles :
print(f"la distribution des valeurs de sentiment: \n{df_data['sentiment'].value_counts()}")

la distribution des valeurs de sentiment: 
positive    25000
negative    25000
Name: sentiment, dtype: int64


# Pre-processing

In [8]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from nltk.stem import PorterStemmer

In [10]:
# Nettoyer et prétraiter les textes (enlever la ponctuation, convertir en minuscules, etc.).
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
def clean_text(text):
    # Supprimer la ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]
    # Appliquer la racinisation (stemming)
    words = [stemmer.stem(word) for word in words]
    # Rejoindre les mots pour former le texte traité
    text = ' '.join(words)
    return text

In [11]:
# encodage
df_data['encoded_sentiment'] = df_data['sentiment'].map({'positive': 1, 'negative': 0})

In [12]:
df_data['cleaned_review'] = df_data['review'].apply(clean_text)

In [13]:
df_data.head()

,review,sentiment,encoded_sentiment,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,1,one review mention watch 1 oz episod youll hoo...
1,A wonderful little production. <br /><br />The...,positive,1,wonder littl product br br film techniqu unass...
2,I thought this was a wonderful way to spend ti...,positive,1,thought wonder way spend time hot summer weeke...
3,Basically there's a family where a little boy ...,negative,0,basic there famili littl boy jake think there ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,petter mattei love time money visual stun film...


In [14]:
# remove unused columns
df_data = df_data[['cleaned_review', 'encoded_sentiment']]

In [15]:
df_data.head()

,cleaned_review,encoded_sentiment
0,one review mention watch 1 oz episod youll hoo...,1
1,wonder littl product br br film techniqu unass...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic there famili littl boy jake think there ...,0
4,petter mattei love time money visual stun film...,1


In [16]:
# Tokenizing the text
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_data['cleaned_review'])

In [17]:
# converting text to sequences
sequences = tokenizer.texts_to_sequences(df_data['cleaned_review'])

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
max_length = 200
vocab_size = 10000
sequence_length = 50

In [20]:
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

In [21]:
# Displaying the first sequence as an example
print("Original text:\n", df_data['cleaned_review'][0])
print("\nTokenized sequence:\n", sequences[0])
print("\nPadded sequence:\n", padded_sequences[0])
print("\nLen Padded sequence:\n", len(padded_sequences[0]))

Original text:
 one review mention watch 1 oz episod youll hook right exactli happen mebr br first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use wordbr br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awaybr br would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti surreal couldnt say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison ex

# Train and Test Data

In [22]:
x = df_data['cleaned_review']
y = df_data['encoded_sentiment']

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(x)

# Convertir les textes en séquences d'entiers
sequences = tokenizer.texts_to_sequences(x)
# Remplir les séquences pour qu'elles aient toutes la même longueur
x_padded = pad_sequences(sequences, maxlen=sequence_length, padding='post', truncating='post')
# Diviser les données en ensembles d'entraînement et de test
x_train, x_test, y_train, y_test = train_test_split(x_padded, y, test_size=0.2, random_state=42)

# 4.	Construction du Modèle LSTM

In [23]:
def create_model(vocab_size=vocab_size, embedding_dim=50, sequence_length=sequence_length, lstm_out=32, dense_units=1, activation='sigmoid', extra_dense_layer=True):
  """
  lstm_units: nombre de neurones LSTM dans la couche LSTM, plus il est élevé, plus d'apprentissage complexe
  embedding_dim: Dim de l'espace d'incorporation
  input_length: longueur des séquences en entrée
  activation: la fonction appliquée à la sortie 'sigmoid' est souvent utilisé pour produire des valuers entre 0 et 1 pour la probabilité de la classe positive.
  """
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length))
  model.add(Bidirectional(LSTM(lstm_out)))
  if extra_dense_layer:
    model.add(Dense(10, activation='relu'))
  model.add(Dense(units=dense_units, activation=activation))
  return model

def compile_model(model):
  # Compiler le modèle
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

def fit_model(model):
  # Entraîner le modèle
  model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))
  return model

def evaluate_model(model):
  loss, accuracy = model.evaluate(x_test, y_test, verbose=2)
  print('---------------Loss - Accuracy----------------')
  print(f'Loss on test set: {loss:.4f}')
  print(f'Accuracy on test set: {accuracy:.4f}')
  y_pred = model.predict(x_test)
  y_pred_binary = (y_pred > 0.5).astype(int)
  precision = accuracy_score(y_test, y_pred_binary)
  print('---------------Precision----------------')
  print(f'Precision on test set: {precision:.4f}')
  print('---------------Classification Report----------------')
  print(classification_report(y_test, y_pred_binary))

In [24]:
model = create_model()
model = compile_model(model)
print('---------------Summary----------------')
print(model.summary())
print('---------------Training----------------')
model = fit_model(model)
print('---------------Evaluation----------------')
evaluate_model(model)

---------------Summary----------------
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            500000    
                                                                 
 bidirectional (Bidirection  (None, 64)                21248     
 al)                                                             
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 521909 (1.99 MB)
Trainable params: 521909 (1.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
---------------Training----------------
Ep

In [25]:
model = create_model(lstm_out=64)
model = compile_model(model)
print('---------------Summary----------------')
print(model.summary())
print('---------------Training----------------')
model = fit_model(model)
print('---------------Evaluation----------------')
evaluate_model(model)

---------------Summary----------------
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 50)            500000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               58880     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 560181 (2.14 MB)
Trainable params: 560181 (2.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
---------------Training----------------


In [26]:
model = create_model(activation='tanh')
model = compile_model(model)
print('---------------Summary----------------')
print(model.summary())
print('---------------Training----------------')
model = fit_model(model)
print('---------------Evaluation----------------')
evaluate_model(model)

---------------Summary----------------
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            500000    
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                21248     
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 521909 (1.99 MB)
Trainable params: 521909 (1.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
---------------Training----------------


# Interpretation des résultats